# Predicción de Fallas con Decision Tree y LightGBM

Este notebook aplica modelos de clasificación para predecir fallas usando datos de la tabla `faliure_probability_base`.

**Características del dataset:**
- Granularidad diaria (`reading_date`)
- Variable objetivo: `faliure` (booleano indicando si hay falla en los próximos 7 días)

**Modelos utilizados:**
- Árbol de Decisión (Decision Tree)
- LightGBM

Este notebook cubre:
- Carga de datos desde MySQL
- Preparación y feature engineering
- Entrenamiento de modelos
- Evaluación y métricas
- Análisis de importancia de características
- Predicciones y análisis de resultados
- Guardar y cargar modelos

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from mysql.connector import Error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    roc_auc_score, roc_curve
)
from lightgbm import LGBMClassifier
import joblib
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Librerías importadas correctamente")

## 2. Carga de Datos desde MySQL

In [ ]:
# Configuración de la base de datos
DB_CONFIG = {
    'host': '127.0.0.1',
    'database': 'palantir_maintenance',
    'user': 'root',
    'password': 'admin',
    'port': 3306
}

def cargar_datos():
    """Cargar datos de la tabla faliure_probability_base (misma función que otros notebooks)"""
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            query = """
            SELECT * 
            FROM faliure_probability_base
            ORDER BY asset_id, reading_date
            """
            df = pd.read_sql(query, connection)
            connection.close()
            print(f"✅ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
            return df
    except Error as e:
        print(f"❌ Error al conectar: {e}")
        return None

# Cargar datos (usando la misma función que los demás notebooks)
df = cargar_datos()

if df is not None and len(df) > 0:
    print(f"\nColumnas disponibles: {list(df.columns)}")
    print(f"\nPrimeras filas:")
    display(df.head())
    print(f"\nInformación del DataFrame:")
    print(df.info())
else:
    print("⚠️ No se pudieron cargar los datos. Verifica la conexión a la base de datos.")

## 3. Análisis Exploratorio de Datos (EDA)

In [ ]:
if df is not None and 'faliure' in df.columns:
    # Análisis de la variable objetivo
    print("Distribución de la variable objetivo 'faliure':")
    faliure_dist = df['faliure'].value_counts()
    print(faliure_dist)
    print(f"\nTasa de fallas: {df['faliure'].mean()*100:.2f}%")
    
    # Visualización
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Gráfico 1: Distribución de fallas
    axes[0, 0].bar(['Sin Falla', 'Con Falla'], faliure_dist.values, color=['green', 'red'])
    axes[0, 0].set_title('Distribución de Fallas en Próximos 7 Días')
    axes[0, 0].set_ylabel('Frecuencia')
    
    # Gráfico 2: Fallas por activo
    fallas_por_activo = df.groupby('asset_id')['faliure'].sum()
    axes[0, 1].bar(fallas_por_activo.index, fallas_por_activo.values)
    axes[0, 1].set_title('Fallas por Activo')
    axes[0, 1].set_xlabel('Asset ID')
    axes[0, 1].set_ylabel('Número de días con falla próxima')
    
    # Gráfico 3: Distribución de sensor_warning_count
    if 'sensor_warning_count_30d' in df.columns:
        df['sensor_warning_count_30d'].fillna(0).hist(bins=20, ax=axes[1, 0])
        axes[1, 0].set_title('Distribución de Alertas de Sensor (30 días)')
        axes[1, 0].set_xlabel('Conteo de Alertas')
        axes[1, 0].set_ylabel('Frecuencia')
    
    # Gráfico 4: Distribución de failure_count
    if 'failure_count_365d' in df.columns:
        df['failure_count_365d'].fillna(0).hist(bins=20, ax=axes[1, 1])
        axes[1, 1].set_title('Distribución de Fallas (365 días)')
        axes[1, 1].set_xlabel('Conteo de Fallas')
        axes[1, 1].set_ylabel('Frecuencia')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No hay datos o columna 'faliure' no encontrada")

## 4. Preparación de Características

In [ ]:
if df is not None and 'faliure' in df.columns:
    # Columnas a excluir
    exclude_cols = ['base_id', 'asset_id', 'reading_date', 'faliure', 
                    'asset_status', 'created_at', 'updated_at']
    
    # Seleccionar características numéricas
    feature_columns = [col for col in df.columns 
                       if col not in exclude_cols and df[col].dtype in ['int64', 'float64']]
    
    print(f"Características seleccionadas: {len(feature_columns)}")
    print(f"\nColumnas: {feature_columns}")
    
    # Preparar datos
    X = df[feature_columns].fillna(0).replace([np.inf, -np.inf], 0)
    y = df['faliure'].astype(int)
    
    print(f"\nForma de X: {X.shape}")
    print(f"Forma de y: {y.shape}")
    print(f"\nDistribución de clases:")
    print(y.value_counts())
    
    if len(X) < 50:
        print("⚠️ ADVERTENCIA: Muy pocos datos para entrenar un modelo robusto")
    else:
        print(f"\n✅ Datos suficientes para entrenamiento: {len(X)} muestras")

## 5. División de Datos en Train y Test

In [ ]:
if 'X' in locals() and 'y' in locals() and len(np.unique(y)) > 1:
    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"Datos de entrenamiento: {X_train.shape}")
    print(f"Datos de prueba: {X_test.shape}")
    
    print(f"\nDistribución de clases en entrenamiento:")
    print(f"  Sin falla: {(y_train == 0).sum()} ({(y_train == 0).sum()/len(y_train)*100:.1f}%)")
    print(f"  Con falla: {(y_train == 1).sum()} ({(y_train == 1).sum()/len(y_train)*100:.1f}%)")
    
    print(f"\nDistribución de clases en prueba:")
    print(f"  Sin falla: {(y_test == 0).sum()} ({(y_test == 0).sum()/len(y_test)*100:.1f}%)")
    print(f"  Con falla: {(y_test == 1).sum()} ({(y_test == 1).sum()/len(y_test)*100:.1f}%)")
    
    # Escalar características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convertir a DataFrame para LightGBM
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
    print("\n✅ Datos divididos y escalados correctamente")
else:
    print("⚠️ No hay datos preparados o solo hay una clase")

## 6. Entrenamiento del Modelo Decision Tree

In [ ]:
if 'X_train' in locals():
    print("="*60)
    print("ENTRENANDO ÁRBOL DE DECISIÓN")
    print("="*60)
    
    # Crear modelo
    dt_model = DecisionTreeClassifier(
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        class_weight='balanced',
        random_state=42
    )
    
    # Entrenar
    dt_model.fit(X_train_scaled, y_train)
    
    # Predicciones
    y_pred_dt = dt_model.predict(X_test_scaled)
    y_pred_proba_dt = dt_model.predict_proba(X_test_scaled)[:, 1]
    
    # Métricas
    print(f"\nAccuracy:  {accuracy_score(y_test, y_pred_dt):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred_dt, zero_division=0):.4f}")
    print(f"Recall:    {recall_score(y_test, y_pred_dt, zero_division=0):.4f}")
    print(f"F1-Score:  {f1_score(y_test, y_pred_dt, zero_division=0):.4f}")
    
    if len(np.unique(y_test)) > 1:
        print(f"AUC-ROC:   {roc_auc_score(y_test, y_pred_proba_dt):.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_dt, 
                                target_names=['Sin Falla', 'Con Falla'],
                                zero_division=0))
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred_dt)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Sin Falla', 'Con Falla'],
                yticklabels=['Sin Falla', 'Con Falla'])
    plt.title('Matriz de Confusión - Árbol de Decisión')
    plt.ylabel('Verdadero')
    plt.xlabel('Predicho')
    plt.show()
    
    print("\n✅ Árbol de Decisión entrenado correctamente")

## 7. Entrenamiento del Modelo LightGBM

In [ ]:
if 'X_train' in locals():
    print("="*60)
    print("ENTRENANDO LIGHTGBM")
    print("="*60)
    
    # Calcular peso para datos desbalanceados
    n_neg = np.sum(y_train == 0)
    n_pos = np.sum(y_train == 1)
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1
    
    print(f"Scale pos weight: {scale_pos_weight:.2f}")
    
    # Crear modelo
    lgbm_model = LGBMClassifier(
        objective='binary',
        metric='binary_logloss',
        boosting_type='gbdt',
        num_leaves=31,
        learning_rate=0.05,
        n_estimators=100,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        verbose=-1
    )
    
    # Entrenar con early stopping
    lgbm_model.fit(
        X_train_scaled_df, y_train,
        eval_set=[(X_test_scaled_df, y_test)],
        eval_names=['valid'],
        callbacks=[
            lgbm_model.early_stopping(stopping_rounds=20, verbose=False)
        ]
    )
    
    # Predicciones
    y_pred_lgbm = lgbm_model.predict(X_test_scaled_df)
    y_pred_proba_lgbm = lgbm_model.predict_proba(X_test_scaled_df)[:, 1]
    
    # Métricas
    print(f"\nAccuracy:  {accuracy_score(y_test, y_pred_lgbm):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred_lgbm, zero_division=0):.4f}")
    print(f"Recall:    {recall_score(y_test, y_pred_lgbm, zero_division=0):.4f}")
    print(f"F1-Score:  {f1_score(y_test, y_pred_lgbm, zero_division=0):.4f}")
    
    if len(np.unique(y_test)) > 1:
        print(f"AUC-ROC:   {roc_auc_score(y_test, y_pred_proba_lgbm):.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_lgbm, 
                                target_names=['Sin Falla', 'Con Falla'],
                                zero_division=0))
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred_lgbm)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
                xticklabels=['Sin Falla', 'Con Falla'],
                yticklabels=['Sin Falla', 'Con Falla'])
    plt.title('Matriz de Confusión - LightGBM')
    plt.ylabel('Verdadero')
    plt.xlabel('Predicho')
    plt.show()
    
    print("\n✅ LightGBM entrenado correctamente")

## 8. Importancia de Características

In [ ]:
if 'lgbm_model' in locals():
    # Importancia de características LightGBM
    lgbm_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': lgbm_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Importancia de características Decision Tree
    dt_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': dt_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Decision Tree
    sns.barplot(data=dt_importance.head(15), x='importance', y='feature', ax=axes[0], palette='Blues_r')
    axes[0].set_title('Top 15 Características - Árbol de Decisión')
    axes[0].set_xlabel('Importancia')
    
    # LightGBM
    sns.barplot(data=lgbm_importance.head(15), x='importance', y='feature', ax=axes[1], palette='Greens_r')
    axes[1].set_title('Top 15 Características - LightGBM')
    axes[1].set_xlabel('Importancia')
    
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 características más importantes (LightGBM):")
    print(lgbm_importance.head(10).to_string(index=False))

## 9. Comparación de Modelos

In [ ]:
if 'y_pred_dt' in locals() and 'y_pred_lgbm' in locals():
    # Comparar modelos
    resultados = []
    
    for nombre, (y_pred, y_proba) in [('Árbol de Decisión', (y_pred_dt, y_pred_proba_dt)),
                                       ('LightGBM', (y_pred_lgbm, y_pred_proba_lgbm))]:
        result = {
            'Modelo': nombre,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred, zero_division=0),
            'Recall': recall_score(y_test, y_pred, zero_division=0),
            'F1-Score': f1_score(y_test, y_pred, zero_division=0)
        }
        if len(np.unique(y_test)) > 1:
            result['AUC-ROC'] = roc_auc_score(y_test, y_proba)
        resultados.append(result)
    
    df_resultados = pd.DataFrame(resultados)
    
    print("="*70)
    print("COMPARACIÓN DE MODELOS")
    print("="*70)
    print(df_resultados.round(4).to_string(index=False))
    
    # Visualización
    fig, ax = plt.subplots(figsize=(12, 6))
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    if 'AUC-ROC' in df_resultados.columns:
        metrics.append('AUC-ROC')
    df_resultados.set_index('Modelo')[metrics].plot(kind='bar', ax=ax)
    plt.title('Comparación de Métricas entre Modelos')
    plt.ylabel('Score')
    plt.xticks(rotation=0)
    plt.legend(title='Métricas', loc='lower right')
    plt.ylim(0, 1.1)
    plt.tight_layout()
    plt.show()
    
    # Curvas ROC
    if len(np.unique(y_test)) > 1:
        plt.figure(figsize=(8, 6))
        fpr_dt, tpr_dt, _ = roc_curve(y_test, y_pred_proba_dt)
        fpr_lgbm, tpr_lgbm, _ = roc_curve(y_test, y_pred_proba_lgbm)
        
        plt.plot(fpr_dt, tpr_dt, label=f'Árbol de Decisión (AUC={roc_auc_score(y_test, y_pred_proba_dt):.3f})')
        plt.plot(fpr_lgbm, tpr_lgbm, label=f'LightGBM (AUC={roc_auc_score(y_test, y_pred_proba_lgbm):.3f})')
        plt.plot([0, 1], [0, 1], 'k--', label='Random')
        
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Curvas ROC')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

## 10. Guardar Modelos

In [ ]:
if 'dt_model' in locals() and 'lgbm_model' in locals():
    # Guardar modelos
    joblib.dump(dt_model, 'decision_tree_failure_model.pkl')
    joblib.dump(lgbm_model, 'lightgbm_failure_model.pkl')
    joblib.dump(scaler, 'feature_scaler.pkl')
    
    # Guardar información de características
    import json
    feature_info = {
        'feature_names': list(X_train.columns),
        'num_features': len(X_train.columns)
    }
    with open('model_feature_info.json', 'w') as f:
        json.dump(feature_info, f, indent=2)
    
    print("✅ Modelos guardados:")
    print("  - decision_tree_failure_model.pkl")
    print("  - lightgbm_failure_model.pkl")
    print("  - feature_scaler.pkl")
    print("  - model_feature_info.json")
    
    # Seleccionar mejor modelo
    dt_f1 = f1_score(y_test, y_pred_dt, zero_division=0)
    lgbm_f1 = f1_score(y_test, y_pred_lgbm, zero_division=0)
    
    if lgbm_f1 >= dt_f1:
        print(f"\n🏆 Mejor modelo: LightGBM (F1-Score: {lgbm_f1:.4f})")
    else:
        print(f"\n🏆 Mejor modelo: Árbol de Decisión (F1-Score: {dt_f1:.4f})")

## 11. Función para Predicciones en Nuevos Datos

In [ ]:
def predecir_fallas(nuevos_datos, modelo, scaler, feature_names):
    """
    Hacer predicciones de fallas en nuevos datos
    
    Parámetros:
    - nuevos_datos: DataFrame con las características
    - modelo: Modelo entrenado
    - scaler: Escalador ajustado
    - feature_names: Lista de nombres de características
    
    Retorna:
    - predicciones: Array con las clases predichas
    - probabilidades: Array con las probabilidades de falla
    """
    # Preparar datos
    datos_preparados = nuevos_datos[feature_names].copy()
    datos_preparados = datos_preparados.fillna(0).replace([np.inf, -np.inf], 0)
    
    # Escalar
    datos_scaled = scaler.transform(datos_preparados)
    datos_scaled_df = pd.DataFrame(datos_scaled, columns=feature_names)
    
    # Predecir
    predicciones = modelo.predict(datos_scaled_df)
    probabilidades = modelo.predict_proba(datos_scaled_df)[:, 1]
    
    return predicciones, probabilidades

# Ejemplo de uso
if 'lgbm_model' in locals() and 'X_test' in locals():
    print("=" * 60)
    print("EJEMPLO DE PREDICCIÓN EN NUEVOS DATOS")
    print("=" * 60)
    
    ejemplo_datos = X_test.head(5)
    pred, prob = predecir_fallas(ejemplo_datos, lgbm_model, scaler, list(X_train.columns))
    
    for i in range(len(pred)):
        estado = "CON FALLA PRÓXIMA" if pred[i] == 1 else "Sin falla próxima"
        print(f"\nMuestra {i+1}: {estado}")
        print(f"  Probabilidad de falla: {prob[i]:.4f}")

## Resumen

En este notebook hemos:

1. ✅ Cargado datos de la tabla `faliure_probability_base` (granularidad diaria)
2. ✅ Analizado la variable objetivo `faliure` (falla en próximos 7 días)
3. ✅ Preparado características para entrenamiento
4. ✅ Entrenado modelo **Árbol de Decisión**
5. ✅ Entrenado modelo **LightGBM**
6. ✅ Comparado métricas de ambos modelos
7. ✅ Visualizado importancia de características
8. ✅ Guardado modelos para uso en producción
9. ✅ Creado función para predicciones en nuevos datos

**Los modelos están listos para predecir fallas en activos de mantenimiento.**